In [ ]:
import carla
import numpy as np
import cv2

In [ ]:
import random
import time
from keras.models import load_model
from agents.navigation.behavior_agent import BehaviorAgent

In [ ]:
client=carla.Client("localhost",2000)
world=client.get_world()

In [ ]:
blueprint_library = world.get_blueprint_library()
spawnpoints=world.get_map().get_spawn_points()

In [ ]:
ar=blueprint_library.find("vehicle.mercedes.coupe_2020")
transforms=random.choice(spawnpoints)
car=world.try_spawn_actor(ar,transforms)

In [ ]:
spectator=world.get_spectator()
x=carla.Transform(car.get_transform().transform(carla.Location(x=-0.4,z=1.8)),car.get_transform().rotation)
spectator.set_transform(x)

In [ ]:
cam=blueprint_library.find("sensor.camera.rgb")
cam.set_attribute("image_size_x","1920")
cam.set_attribute("image_size_y","1080")
cam.set_attribute("fov","110")
cam.set_attribute("sensor_tick",".2")
transforms=carla.Transform(carla.Location(z=1.8,x=-.2))
camera=world.spawn_actor(cam,transforms,attach_to=car)
time.sleep(.2)


In [ ]:
def show_me(image,camera_data):
    r=np.array(image.raw_data)
    r1=r.reshape((1080,1920,4))
    r2=r1[:,:,:3]
    im=cv2.cvtColor(r2,cv2.COLOR_BGR2YUV)
    im=cv2.GaussianBlur(im,(3,3),0)
    im=cv2.resize(im,(400,225))
    #im1=np.array([im])
    im1=np.array([im])
    camera_data["zs"]=im1
    

In [ ]:
image_w=cam.get_attribute("image_size_x").as_int()
image_h=cam.get_attribute("image_size_y").as_int()
camera_data={"zs":np.zeros((image_h,image_w,3))}
camera.listen(lambda image:show_me(image,camera_data))

In [ ]:
strmodel=load_model("C:\\Carla\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\examples\\behaviour cloning\\model_a.h5")
thrmodel=load_model("C:\\Carla\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\examples\\behaviour cloning\\model_b.h5")
brkmodel=load_model("C:\\Carla\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\examples\\behaviour cloning\\model_c.h5")

In [ ]:
agent=BehaviorAgent(car,behavior="normal")
time_pres=time.time()
st=0
br=0
thr=0
while True:
    if agent.done():
        agent.set_destination(random.choice(spawnpoints).location)
        print("Target has been reached, searching for next target")
    steer1=float(strmodel.predict(camera_data["zs"]))
    throttle1=float(thrmodel.predict(camera_data["zs"]))
    brake1=float(brkmodel.predict(camera_data["zs"]))
    if thr<=throttle1:
        control=carla.VehicleControl(throttle=throttle1)
        thr=throttle1
        st=steer1
    elif thr>throttle1 and steer1>=st or steer1<=st:
        control=carla.VehicleControl(steer=steer1)
        thr=throttle1
        st=steer1
    else:
        control=carla.VehicleControl(brake=brake1)
        thr=throttle1
        st=steer1

    print(control)
    car.apply_control(control)
